In [43]:
from graphdatascience import GraphDataScience
from getpass import getpass
import pandas as pd

# Set up connections

In [3]:
vertex_ai_token = getpass()

 ········


In [4]:
neo4j_password = getpass()

 ········


In [5]:
neo4j_uri = "neo4j+s://bc9f751a.databases.neo4j.io"
neo4j_user = "neo4j"
gds = GraphDataScience(neo4j_uri, auth=(neo4j_user, neo4j_password))

# Embed summaries using same model as documents

In [6]:
gds.run_cypher("""
CREATE VECTOR INDEX topic_group_summary_embeddings IF NOT EXISTS 
                  FOR (g:TopicGroup)
                  ON (g.summaryEmbedding)
                  OPTIONS {indexConfig: 
                      {`vector.dimensions`: 768,
                       `vector.similarity_function`: 'cosine'
                       }}
                       """)

""


In [7]:
gds.run_cypher("""
MATCH (g:TopicGroup)
CALL {
WITH g
WITH collect(g) AS groupNodes, collect(g.summary) AS summaries
CALL genai.vector.encodeBatch(summaries, "VertexAI", { token: $token, projectId: $project }) 
YIELD index, vector
WITH groupNodes[index] AS g, vector
CALL db.create.setNodeVectorProperty(g, 'summaryEmbedding', vector)
} IN TRANSACTIONS Of 1000 ROWS""",
              {"token": vertex_ai_token, "project": "neo4j-ps-202001"})


""


# Rank summaries for retrieval

In [34]:
def rank_relationships(alpha, max_rank, property_name):
    gds.run_cypher("""
    CALL apoc.periodic.iterate(
    "MATCH (g:TopicGroup)
    RETURN g.id AS groupId",
    "MATCH (g:TopicGroup {id:groupId})<-[:IN_GROUP]-()<-[:HAS_TOPIC]-(d)
    WITH g, collect(distinct d) AS docList
    WITH g, docList, 
    //Create a map {doc.index: cosine-similarity-to-summary}
    apoc.map.fromPairs(
        [d in docList | 
            [toString(d.index), gds.similarity.cosine(d.embedding, g.summaryEmbedding)]
        ]
    ) AS summaryMap
    WITH 
    g, docList, summaryMap,
    //Create a map {doc.index: {otherDocIndex: cosine-similarity-to-ther-doc}}
    [doc in docList |
        [toString(doc.index), 
            apoc.map.fromPairs(
                [d in docList where d <> doc | 
                [toString(d.index), gds.similarity.cosine(d.embedding, doc.embedding)] ]
            )
        ]
    ] AS neighborMapList
    WITH g, docList, summaryMap, apoc.map.fromPairs(neighborMapList) AS neighborMap
    WITH g,
    //Calculate rank
    REDUCE(rankList=[], x in docList |
    CASE WHEN size(rankList) < $maxRank THEN
    rankList + apoc.coll.sortMaps(
        [doc IN docList WHERE NOT doc IN rankList | 
        {node: doc,
        score: $alpha * summaryMap[toString(doc.index)] +
         (1-$alpha) * (1 - coalesce(apoc.coll.max([
             rankedDoc in rankList | neighborMap[toString(doc.index)][toString(rankedDoc.index)]
         ]), 0))}
        ], 'score'
    )[0]['node']
    ELSE rankList END
    ) AS rankedList
    UNWIND range(0, size(rankedList)-1) AS idx
    WITH g,
    idx + 1 AS rank, 
    rankedList[idx] AS d
    MATCH (g)<-[:IN_GROUP]-()<-[h:HAS_TOPIC]-(d)
    WITH rank, d, g, collect(h) AS rels
    WITH rank, rels[0] AS firstRel
    CALL apoc.create.setRelProperty(firstRel, $propertyName, rank)
    YIELD rel
    RETURN count(*) AS relCount",
    {params: {alpha: $alpha, maxRank: $maxRank, propertyName: $propertyName},
    batchSize: 50,
    parallel:true,
    retries:1})
    """, {"alpha":alpha, "maxRank": max_rank, "propertyName": property_name})

In [35]:
rank_relationships(0.25, 50, "rankAlpha25")

In [36]:
rank_relationships(0.5, 50, "rankAlpha50")

In [37]:
rank_relationships(0.75, 50, "rankAlpha75")

In [38]:
rank_relationships(0.05, 50, "rankAlpha05")

In [39]:
rank_relationships(0.95, 50, "rankAlpha95")

In [40]:
def compare_ranks(group_id):
    rank_df = gds.run_cypher("""
    MATCH (d)-[h:HAS_TOPIC]->()-[:IN_GROUP]->(g:TopicGroup {id:$groupId})
    WITH g, h, d
    ORDER BY h.rankAlpha05
    WITH g, collect(h) AS rels, collect(d) AS docs, collect(d.text)[..5] AS rank05
    UNWIND range(0, size(rels)-1) AS idx
    WITH g, rank05, docs[idx] AS d, rels[idx] AS h
    ORDER BY h.rankAlpha25
    WITH g,  collect(h) as rels, collect(d) AS docs, rank05, collect(d.text)[..5] AS rank25
    UNWIND range(0, size(rels)-1) AS idx
    WITH g, rank05, rank25, docs[idx] AS d, rels[idx] AS h
    ORDER BY h.rankAlpha50
    WITH g,  collect(h) as rels, collect(d) AS docs, rank05, rank25, collect(d.text)[..5] AS rank50
    UNWIND range(0, size(rels)-1) AS idx
    WITH g, rank05, rank25, rank50, docs[idx] AS d, rels[idx] AS h
    ORDER BY h.rankAlpha75
    WITH g,  collect(h) as rels, collect(d) AS docs, rank05, rank25, rank50, collect(d.text)[..5] AS rank75
    UNWIND range(0, size(rels)-1) AS idx
    WITH g, rank05, rank25, rank50, rank75, docs[idx] AS d, rels[idx] AS h
    ORDER BY h.rankAlpha95
    WITH g,  collect(h) as rels, collect(d) AS docs, rank05, rank25, rank50, rank75, collect(d.text)[..5] AS rank95
    UNWIND range(0, 4) AS idx
    RETURN g.descriptions as descriptions, g.summary AS summary, idx + 1 as rank, 
    rank05[idx] AS alpha05, 
    rank25[idx] AS alpha25, 
    rank50[idx] AS alpha50,
    rank75[idx] AS alpha75, 
    rank95[idx] AS alpha95
    ORDER BY rank""", {"groupId": group_id})
    return rank_df


In [44]:
pd.set_option('display.max_colwidth', None)

In [45]:
compare_ranks(2)

,descriptions,summary,rank,alpha05,alpha25,alpha50,alpha75,alpha95
0,"[All pairs shortest path, Average shortest path, Measuring shortest path, Shortest Path AStar, Shortest path, Shortest path guarantee, Shortest path use case, Shortest weighted path, Single shortest path, Stops at shortest path, shortestPath deprecation, shortestPath function, weighted shortest path]","The common themes among the topics include finding the shortest path between nodes in a graph, utilizing algorithms like AStar and Dijkstra for calculating shortest paths, and handling weighted relationships to determine the shortest path based on specific criteria. These topics often involve optimizing performance, defining path patterns, and considering different use cases for finding the shortest path in a graph.",1,"All Pairs Shortest Path The All Pairs Shortest Path (APSP) calculates the shortest (weighted) path between all pairs of nodes. This algorithm has optimizations that make it quicker than calling the Single Source Shortest Path algorithm for every pair of nodes in the graph. This feature is in the alpha tier. For more information on feature tiers, see API Tiers. Supported algorithm traits: Directed Undirected Heterogeneous nodes Heterogeneous relationships Weighted relationships Glossary Directed","All Pairs Shortest Path The All Pairs Shortest Path (APSP) calculates the shortest (weighted) path between all pairs of nodes. This algorithm has optimizations that make it quicker than calling the Single Source Shortest Path algorithm for every pair of nodes in the graph. This feature is in the alpha tier. For more information on feature tiers, see API Tiers. Supported algorithm traits: Directed Undirected Heterogeneous nodes Heterogeneous relationships Weighted relationships Glossary Directed","All Pairs Shortest Path The All Pairs Shortest Path (APSP) calculates the shortest (weighted) path between all pairs of nodes. This algorithm has optimizations that make it quicker than calling the Single Source Shortest Path algorithm for every pair of nodes in the graph. This feature is in the alpha tier. For more information on feature tiers, see API Tiers. Supported algorithm traits: Directed Undirected Heterogeneous nodes Heterogeneous relationships Weighted relationships Glossary Directed","All Pairs Shortest Path The All Pairs Shortest Path (APSP) calculates the shortest (weighted) path between all pairs of nodes. This algorithm has optimizations that make it quicker than calling the Single Source Shortest Path algorithm for every pair of nodes in the graph. This feature is in the alpha tier. For more information on feature tiers, see API Tiers. Supported algorithm traits: Directed Undirected Heterogeneous nodes Heterogeneous relationships Weighted relationships Glossary Directed","All Pairs Shortest Path The All Pairs Shortest Path (APSP) calculates the shortest (weighted) path between all pairs of nodes. This algorithm has optimizations that make it quicker than calling the Single Source Shortest Path algorithm for every pair of nodes in the graph. This feature is in the alpha tier. For more information on feature tiers, see API Tiers. Supported algorithm traits: Directed Undirected Heterogeneous nodes Heterogeneous relationships Weighted relationships Glossary Directed"
1,"[All pairs shortest path, Average shortest path, Measuring shortest path, Shortest Path AStar, Shortest path, Shortest path guarantee, Shortest path use case, Shortest weighted path, Single shortest path, Stops at shortest path, shortestPath deprecation, shortestPath function, weighted shortest path]","The common themes among the topics include finding the shortest path between nodes in a graph, utilizing algorithms like AStar and Dijkstra for calculating shortest paths, and handling weighted relationships to determine the shortest path based on specific criteria. These topics often involve optimizing performance, defining path patterns, and considering different use cases for find

In [47]:
compare_ranks(123)

,descriptions,summary,rank,alpha05,alpha25,alpha50,alpha75,alpha95
0,"[Immediate request retry, Limited retries, MAP parameter, Path parameter in functions, address parameter, maximum retry time per request, path PATH?, paths [Path] parameter]","The common themes among the topics include handling retries for requests, specifying parameters in functions and procedures, and utilizing paths or path-related parameters in various operations within Neo4j. These topics also touch on addressing specific configurations, such as maximum retry times per request and utilizing MAP parameters for customization. Additionally, the topics discuss scenarios like immediate request retries and deadlock handling strategies within Neo4j software.",1,"Deadlock handling in code When dealing with deadlocks in code, there are several issues you may want to address: Only do a limited amount of retries, and fail if a threshold is reached. Pause between each attempt to allow the other transaction to finish before trying again. A retry loop can be useful not only for deadlocks but for other types of transient errors as well.","Deadlock handling in code When dealing with deadlocks in code, there are several issues you may want to address: Only do a limited amount of retries, and fail if a threshold is reached. Pause between each attempt to allow the other transaction to finish before trying again. A retry loop can be useful not only for deadlocks but for other types of transient errors as well.","Deadlock handling in code When dealing with deadlocks in code, there are several issues you may want to address: Only do a limited amount of retries, and fail if a threshold is reached. Pause between each attempt to allow the other transaction to finish before trying again. A retry loop can be useful not only for deadlocks but for other types of transient errors as well.","Deadlock handling in code When dealing with deadlocks in code, there are several issues you may want to address: Only do a limited amount of retries, and fail if a threshold is reached. Pause between each attempt to allow the other transaction to finish before trying again. A retry loop can be useful not only for deadlocks but for other types of transient errors as well.","Deadlock handling in code When dealing with deadlocks in code, there are several issues you may want to address: Only do a limited amount of retries, and fail if a threshold is reached. Pause between each attempt to allow the other transaction to finish before trying again. A retry loop can be useful not only for deadlocks but for other types of transient errors as well."
1,"[Immediate request retry, Limited retries, MAP parameter, Path parameter in functions, address parameter, maximum retry time per request, path PATH?, paths [Path] parameter]","The common themes among the topics include handling retries for requests, specifying parameters in functions and procedures, and utilizing paths or path-related parameters in various operations within Neo4j. These topics also touch on addressing specific configurations, such as maximum retry times per request and utilizing MAP parameters for customization. Additionally, the topics discuss scenarios like immediate request retries and deadlock handling strategies within Neo4j software.",2,STRING? null config MAP? null Output parameters path PATH? weight FLOAT? More documentation of apoc.algo.aStarConfig apoc.algo.aStar apoc.algo.allSimplePaths ?,STRING? null config MAP? null Output parameters path PATH? weight FLOAT? More documentation of apoc.algo.aStarConfig apoc.algo.aStar apoc.algo.allSimplePaths ?,STRING? null config MAP? null Output parameters path PATH? weight FLOAT? More documentation of apoc.algo.aStarConfig apoc.algo.aStar apoc.algo.allSimplePaths ?,STRING? null config MAP? null Output parameters path PATH? weight FLOAT? More documentation of apoc.algo.aStarConfig apoc.algo.aStar apoc.algo.allSimplePaths ?,STRING? null config MAP? null Output parameters path PATH? weight FLOAT